## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description,Country,Date
0,0,Banikoara,11.2985,2.4386,73.31,13,15,5.28,few clouds,BJ,2022-02-12 00:52:29
1,1,Puerto Ayora,-0.7393,-90.3518,72.55,100,60,1.01,broken clouds,EC,2022-02-12 00:52:29
2,2,Hithadhoo,-0.6000,73.0833,81.14,69,100,8.97,overcast clouds,MV,2022-02-12 00:52:29
3,3,Bonthe,7.5264,-12.5050,76.50,94,8,3.71,clear sky,SL,2022-02-12 00:50:23
4,4,Albany,42.6001,-73.9662,51.80,57,100,1.01,overcast clouds,US,2022-02-12 00:52:30


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("Please enter the minimum temperate for your vacation. "))
max_temp = float(input("Please enter the maximum temperate for your vacation. "))

Please enter the minimum temperate for your vacation. 70
Please enter the maximum temperate for your vacation. 90


In [11]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description,Country,Date
0,0,Banikoara,11.2985,2.4386,73.31,13,15,5.28,few clouds,BJ,2022-02-12 00:52:29
1,1,Puerto Ayora,-0.7393,-90.3518,72.55,100,60,1.01,broken clouds,EC,2022-02-12 00:52:29
2,2,Hithadhoo,-0.6000,73.0833,81.14,69,100,8.97,overcast clouds,MV,2022-02-12 00:52:29
3,3,Bonthe,7.5264,-12.5050,76.50,94,8,3.71,clear sky,SL,2022-02-12 00:50:23
5,5,Cabo San Lucas,22.8909,-109.9124,72.61,66,0,1.01,clear sky,MX,2022-02-12 00:49:31
7,7,Maragogi,-9.0122,-35.2225,79.79,77,27,9.80,scattered clouds,BR,2022-02-12 00:49:48
9,9,Ulladulla,-35.3500,150.4667,72.50,76,84,12.03,broken clouds,AU,2022-02-12 00:52:31
10,10,Soyo,-6.1349,12.3689,76.87,86,33,5.53,scattered clouds,AO,2022-02-12 00:52:31
14,14,Denpasar,-8.6500,115.2167,80.26,88,20,5.75,few clouds,ID,2022-02-12 00:52:32
15,15,Vanimo,-2.6741,141.3028,79.79,86,100,2.71,overcast clouds,PG,2022-02-12 00:52:32


In [23]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather Description    0
Country                2
Date                   0
dtype: int64

In [22]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

clean_df.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather Description    0
Country                0
Date                   0
dtype: int64

In [24]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Banikoara,BJ,73.31,few clouds,11.2985,2.4386,
1,Puerto Ayora,EC,72.55,broken clouds,-0.7393,-90.3518,
2,Hithadhoo,MV,81.14,overcast clouds,-0.6000,73.0833,
3,Bonthe,SL,76.50,clear sky,7.5264,-12.5050,
5,Cabo San Lucas,MX,72.61,clear sky,22.8909,-109.9124,
7,Maragogi,BR,79.79,scattered clouds,-9.0122,-35.2225,
9,Ulladulla,AU,72.50,broken clouds,-35.3500,150.4667,
10,Soyo,AO,76.87,scattered clouds,-6.1349,12.3689,
14,Denpasar,ID,80.26,few clouds,-8.6500,115.2167,
15,Vanimo,PG,79.79,overcast clouds,-2.6741,141.3028,


In [26]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [30]:
#check hotel data, and inspect empty values
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Weather Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [34]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Banikoara,BJ,73.31,few clouds,11.2985,2.4386,Hôtel de l'UCom CVPC
1,Puerto Ayora,EC,72.55,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
2,Hithadhoo,MV,81.14,overcast clouds,-0.6000,73.0833,Scoop Guest House
3,Bonthe,SL,76.50,clear sky,7.5264,-12.5050,Bonthe Holiday Village
5,Cabo San Lucas,MX,72.61,clear sky,22.8909,-109.9124,Hotel Tesoro Los Cabos


In [35]:
# 8a. Create the output File (CSV)
output_data_file = "vacation_search/weatherpy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [36]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Weather Description</dt><dd>{Weather Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [42]:
import matplotlib.pyplot as plt

In [44]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure 
#use the download button at top left of figure to save as png and place in vacation_search folder
fig

Figure(layout=FigureLayout(height='420px'))